<a href="https://colab.research.google.com/github/vzinche/training_ML_for_image_analysis_EBI/blob/master/master_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import os
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from scipy.ndimage import binary_erosion
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms, utils
from tqdm import tqdm

In [0]:
class NucleiDataset(Dataset):
    """ Nuclei and masks """
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = os.listdir(root_dir)
        self.to_tensor = transforms.ToTensor()
        self.inp_transforms = transforms.Compose([transforms.Grayscale(),
                                                  transforms.ToTensor(),
                                                  transforms.Normalize([0.5], [0.5])
                                                  ])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.samples[idx],
                                'images', self.samples[idx]+'.png')
        image = Image.open(img_name)
        image = image.convert("RGB")
        image = self.inp_transforms(image)
        masks_dir = os.path.join(self.root_dir, self.samples[idx], 'masks')
        if not os.path.isdir(masks_dir):
            return image
        masks_list = os.listdir(masks_dir)
        mask = torch.zeros(1, len(image[0]), len(image[0][0]))
        for mask_name in masks_list:
            one_nuclei_mask = Image.open(os.path.join(masks_dir, mask_name))
            one_nuclei_mask = binary_erosion(one_nuclei_mask).astype('float32')
            one_nuclei_mask = self.to_tensor(one_nuclei_mask[..., np.newaxis])
            mask += one_nuclei_mask
        if self.transform is not None:
            image, mask = self.transform([image, mask])
        return image, mask



In [0]:
def focal_loss(prediction, mask, gamma=2):
    loss = F.binary_cross_entropy_with_logits(prediction, mask, reduce=False)
    invprobs = F.logsigmoid(-prediction * (mask * 2 - 1))
    loss = (invprobs * gamma).exp() * loss
    return loss.mean()


def iterate(data):
    return tqdm(data, desc='Samples processed', bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}')


def get_iou(prediction, mask):
    iou = (prediction & mask).sum().float() / (prediction | mask).sum().float()
    return iou


def show_images(inp, mask, pred):
    f, axarr = plt.subplots(1, 3)
    axarr[0].imshow(inp[0][0])
    axarr[1].imshow(mask[0][0])
    axarr[2].imshow(pred[0][0])
    _ = [ax.axis('off') for ax in axarr]
    plt.show()


def show_dataset(dataset):
    idx = np.random.randint(0, len(dataset))
    img, mask = dataset[idx]
    f, axarr = plt.subplots(1, 2)
    axarr[0].imshow(img[0])
    axarr[1].imshow(mask[0])
    _ = [ax.axis('off') for ax in axarr]
    plt.show()


class RandomCrop(object):
    """Crop randomly the input image and the output mask"""
    def __init__(self, crop_size):
        assert isinstance(crop_size, (int, tuple, list))
        if isinstance(crop_size, int):
            self.output_size = (crop_size, crop_size)
        else:
            assert len(crop_size) == 2
            self.crop_size = crop_size

    def __call__(self, sample):
        assert len(sample) == 2
        image, mask = sample
        w, h = image.shape[1:]
        new_w, new_h = self.output_size
        top = np.random.randint(0, h - new_h) if h - new_h > 0 else 0
        left = np.random.randint(0, w - new_w) if w - new_w > 0 else 0
        image = image[:, left: left + new_w, top: top + new_h]
        mask = mask[:, left: left + new_w, top: top + new_h]
        return image, mask


def evaluate(dataloader, model, device='cpu'):
    print ('Evaluating!')
    dataset_size = len(dataloader)
    test_accuracy = 0.0
    test_iou = 0.0
    count = 0
    model.eval()
    with torch.no_grad():
        for images, masks in iterate(dataloader):
            images = images.to(device)
            masks = masks.to(device)
            count += 1
            outputs = model(images)
            predictions = (outputs > 0.5)
            accuracy = torch.mean((predictions == masks.byte()).float())
            iou = get_iou(predictions, masks.type(torch.bool))
            test_accuracy += accuracy.item()
            test_iou += iou.item()
    epoch_accuracy = test_accuracy / dataset_size
    epoch_iou = test_iou / dataset_size
    print('Evaluation iou is {:.6f}, accuracy is {:.6f}'.format(epoch_iou, epoch_accuracy))


In [0]:
class UNetBlock(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(UNetBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, out_dim, kernel_size=3, padding=1)
        self.norm1 = nn.BatchNorm2d(out_dim)
        self.conv2 = nn.Conv2d(out_dim, out_dim, kernel_size=3, padding=1)
        self.norm2 = nn.BatchNorm2d(out_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.activation(x)
        x = self.norm1(x)
        x = self.conv2(x)
        x = self.activation(x)
        x = self.norm2(x)
        return x


class UNetBlockDown(UNetBlock):
    def __init__(self, in_dim, out_dim):
        super(UNetBlockDown, self).__init__(in_dim, out_dim)
        self.pool = nn.MaxPool2d(2, stride=2)

    def forward(self, x):
        x = self.pool(x)
        x = super().forward(x)
        return x


class UNetBlockUp(UNetBlock):
    def __init__(self, in_dim, out_dim):
        super(UNetBlockUp, self).__init__(in_dim, out_dim)
        self.up_conv = nn.ConvTranspose2d(in_dim, out_dim, stride=2, kernel_size=2)
        self.up_norm = nn.BatchNorm2d(out_dim)

    def forward(self, x, saved_x):
        x = self.up_conv(x)
        x = self.activation(x)
        x = self.up_norm(x)
        x = torch.cat((x, saved_x), 1)
        x = super().forward(x)
        return x


class UNet(nn.Module):
    def __init__(self, in_filters, num_layers):
        super(UNet, self).__init__()
        self.first_layer = UNetBlock(1, in_filters)
        self.blocks_down = nn.ModuleList()
        for _ in range(num_layers):
            self.blocks_down.append(UNetBlockDown(in_filters, in_filters*2))
            in_filters *= 2
        self.blocks_up = nn.ModuleList()
        for _ in range(num_layers):
            self.blocks_up.append(UNetBlockUp(in_filters, in_filters//2))
            in_filters //= 2
        self.last_layer = nn.Sequential(
            nn.Conv2d(in_filters, 1, kernel_size=1, padding=0), nn.Sigmoid())

    def forward(self, x):
        saved_x = []
        x = self.first_layer(x)
        for block in self.blocks_down:
            saved_x.append(x)
            x = block(x)
        for i in range(len(self.blocks_up)):
            x = self.blocks_up[i](x, saved_x[-1-i])
        x = self.last_layer(x)
        return x

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1tyI7ig2obOxAdEnKBrUXFD7uZQX9tRKD' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1tyI7ig2obOxAdEnKBrUXFD7uZQX9tRKD" -O nuclei_data.zip && rm -rf /tmp/cookies.txt
!unzip -qq nuclei_data.zip && rm nuclei_data.zip

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1O66UElt2ZfhLXUKKX_nTxmIXh6fMA2rT' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1O66UElt2ZfhLXUKKX_nTxmIXh6fMA2rT" -O kaggle_data.zip && rm -rf /tmp/cookies.txt
!unzip -qq kaggle_data.zip && rm kaggle_data.zip
!mkdir nuclei_train_data && unzip -qq stage1_train.zip -d nuclei_train_data/ && rm stage1_train.zip
!mkdir nuclei_test_data && n=0 && for file in nuclei_train_data/*; do test $n -eq 0 && mv "$file" nuclei_test_data/; n=$((n+1)); n=$((n%4)); done

In [0]:
!ls nuclei_train_data | wc -l && ls nuclei_test_data | wc -l

In [0]:
!ls -ltrh

In [0]:
TRAIN_DATA_PATH = 'nuclei_train_data'
train_data = NucleiDataset(TRAIN_DATA_PATH, RandomCrop(256))
train_dataloader = DataLoader(train_data, batch_size=5, shuffle=True)

In [0]:
show_dataset(train_data)

In [0]:
TEST_DATA_PATH = 'nuclei_test_data'
test_data = NucleiDataset(TEST_DATA_PATH, RandomCrop(256))
test_dataloader = DataLoader(test_data, batch_size=5)

In [0]:
show_dataset(test_data)

In [0]:
NUM_LAYERS = 3    # determines the capacity and the 'depth' (filed of view) of the network
IN_FILTERS = 32   # the number of the feature maps in the first layer - also affects the capacity
model=UNet(IN_FILTERS, NUM_LAYERS)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
def train(model, optimizer, train_loader, test_loader, num_epochs=10, device='cpu', writer=None):
    dataset_size = len(train_loader)  #how many samples we have
    model = model.to(device)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        model.train()
        train_loss = 0.0    # the value that we will backprop
        train_accuracy = 0.0    # just a helpful (for us) metric
        train_iou = 0.0    # another helpful metric
        count = 0
        for images, masks in iterate(train_loader):
            images = images.to(device)
            masks = masks.to(device)
            count += 1
            optimizer.zero_grad()    # erase all the gradient from the previous steps
            outputs = model(images)    # predict
            predictions = (outputs > 0.5)    # binarize the predictions to get a mask
            if count % 10 == 0:    # every tenth iteration show how we are performing
                show_images(images.cpu(), masks.cpu(), predictions.cpu())
            loss = focal_loss(outputs, masks)    # calculate the loss between the predictions and the ground truth
            accuracy = torch.mean((predictions == masks).float())    # how much is the prediction similar to the ground truth? pixelwise
            iou = get_iou(predictions, masks.type(torch.bool))    # calculate the intersection over union (explained below)
            loss.backward()    # compute the gradients for every neuron
            optimizer.step()    # update the weights!
            if writer:
              writer.add_scalar('training loss', loss.item(), epoch * dataset_size + count)
              writer.add_scalar('training accuracy', accuracy.item(), epoch * dataset_size + count)
              writer.add_scalar('training iou', iou.item(), epoch * dataset_size + count)
            train_loss += loss.item()
            train_accuracy += accuracy.item()
            train_iou += iou.item()
        epoch_loss = train_loss / dataset_size
        epoch_accuracy = train_accuracy / dataset_size
        epoch_iou = train_iou / dataset_size
        print('Training loss is {:.6f}, iou is {:.6f}, accuracy is {:.6f}'.format(epoch_loss, epoch_iou, epoch_accuracy))
        evaluate(test_loader, model, device)    # every epoch we want to check how the model performs on a previously unseen data
    return model

In [0]:
writer = SummaryWriter()
#writer.add_graph(model, test_dataloader.__iter__().__next__()[0])

In [0]:
%load_ext tensorboard
%tensorboard --logdir runs/

In [0]:
model = train(model, optimizer, train_dataloader, test_dataloader, num_epochs=10, device=device, writer=writer)